# Robust vehicle pre-allocation

In [1]:
import cvxpy as cp
import scipy as sc
import numpy as np
import numpy.random as npr
import torch
from sklearn import datasets
import pandas as pd
import lropt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1.inset_locator import mark_inset, zoomed_inset_axes

In [2]:
import pandas as pd
data = pd.read_csv('https://xiongpengnus.github.io/rsome/taxi_rain.csv')
demand = data.loc[:, 'Region1':'Region10']  
data = demand.values                              # sample demand as an array
d_ub = demand.max().values                      # upper bound of demand
d_lb = demand.min().values 
# S = dhat.shape[0]  
I, J = 1, 10
r = np.array([4.50, 4.41, 3.61, 4.49, 4.38, 4.58, 4.53, 4.64, 4.58, 4.32])
c = 3 * np.ones(J)
q = 400 * np.ones(I)

y_data = []
for i in range(10):
    y_data.append(r + np.random.normal(0,0.1,J))
y_data = np.array(y_data)

In [3]:
c_us = np.vstack([np.eye(J),-np.eye(J)])
d = np.hstack([d_ub, -d_lb])
u = lropt.UncertainParameter(J,
                        uncertainty_set=lropt.Ellipsoidal(p=2,
                                                    data=data, rho = 20, c=c_us,d=d))
# Formulate the Robust Problem
x = cp.Variable(J)
r = lropt.Parameter(J, data = y_data)
y = cp.Variable(J)
Y = cp.Variable((J,J))
t = cp.Variable()
r.value = y_data[0]
objective = cp.Minimize(t)
constraints = [(c-r)@x + r@y + r@Y@u <= t]
for i in range(J):
    constraints += [y[i] >= x[i] - (Y[i]+np.eye(J)[i])@u]
    constraints += [y[i] + Y[i]@u >= 0]
constraints += [x >= 0, cp.sum(x)<= q]
prob = lropt.RobustProblem(objective=objective, constraints=constraints)

In [4]:
prob.solve()

run 0: test value -62.813, violations 0.0: 100%|██████████| 45/45 [00:11<00:00,  4.08it/s]


-67.25590628061082